# Content generation (image, video, audio)

#### openai/whisper-large-v3-turbo

In [1]:
# Install the transformers package
%pip install transformers
%pip install torchaudio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install librosa soundfile


In [34]:
from transformers import pipeline

# Initialize the pipeline with the Whisper model
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

# Specify your audio file path (wav, mp3, etc.)
audio_file_path = "media/sample-2.mp3"

# Run the speech recognition pipeline
result = pipe(audio_file_path)

# Output the transcription
print("Transcription:", result["text"])


Device set to use cpu
/home/vai/Desktop/chat_bot_experiment_one/venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Transcription:  but the same time safe to scarlet and to a silence which had been friends to that the doctor was put into her thoughts


In [ ]:
result = pipe(audio_file_path, language="bn")  # for Bengali
# or
result = pipe(audio_file_path, language="en")  # for English


In [12]:
%pip install --upgrade soundfile


Note: you may need to restart the kernel to use updated packages.


In [32]:
import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# Load the processor and model
processor = AutoProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en")

# Load the dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

# Extract audio from the dataset and get the sample rate
audio_array = ds[0]["audio"]["array"]
sample_rate = ds[0]["audio"]["sampling_rate"]

# Ensure the waveform is at the required sample rate for Whisper (16 kHz)
if sample_rate != 16000:
    # Optionally resample here if necessary
    import torchaudio
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    audio_array = resampler(torch.tensor(audio_array).unsqueeze(0)).squeeze(0).numpy()
    sample_rate = 16000  # Update sample rate after resampling

# Process the audio with the correct sample rate
inputs = processor(audio_array, sampling_rate=sample_rate, return_tensors="pt", padding=True)
input_features = inputs.input_features

# Generate transcription
generated_ids = model.generate(input_features)

# Decode the generated ids into text
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Output the transcription
print(transcription)


 Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.


#### facebook/wav2vec2-large-960h

In [37]:
from transformers import pipeline

# Initialize the pipeline with the Wav2Vec2 model
pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

# Specify your audio file path (wav, mp3, etc.)
audio_file_path = "media/sample-2.mp3"  # Make sure this is a wav file

# Run the speech recognition pipeline
result = pipe(audio_file_path)

# Output the transcription
print("Transcription:", result["text"])


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Transcription: BUT THE SAME TIME SAFE TO SCARLOT ANTO A SILENCE WHICH HAD BEEN FRIENDS TO THAT THE DOCTOR WAS PUT INTO HER THOUGHTS


#### facebook/wav2vec2-base

In [8]:
model_name = "facebook/wav2vec2-base" 